In [0]:
OUTPUTPath = './output/'
!mkdir ./output

In [2]:
# Save model to your Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import re
import numpy as np

In [110]:
import os
import pandas as pd
DATA_SET_DIR = './gdrive/My Drive/NLP/Final-Project/data/'
TRAINING_DATA_PATH = DATA_SET_DIR+'olid-training-clean.csv'
TEST_PATH_DICT = {}
TEST_A_DATA_PATH = DATA_SET_DIR+'testset-levela-clean.tsv'
TEST_B_DATA_PATH = DATA_SET_DIR+'testset-levelb-clean.tsv'
TEST_C_DATA_PATH = DATA_SET_DIR+'testset-levelc-clean.tsv'
TEST_PATH_DICT['subtask_a'] = TEST_A_DATA_PATH
TEST_PATH_DICT['subtask_b'] = TEST_B_DATA_PATH
TEST_PATH_DICT['subtask_c'] = TEST_C_DATA_PATH

data={}
data['train'] = {}
data['validation'] = {}
data['test'] = {}
task_list = ['subtask_a','subtask_b','subtask_c']
train = pd.read_csv(TRAINING_DATA_PATH,sep='\t', index_col='id')
for subtask in task_list:
    data['test'][subtask] = pd.read_csv(TEST_PATH_DICT[subtask],sep='\t', index_col='id')
train.head(3)

,tweet_exp,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you are drunk!!! @USER #MA...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN


In [111]:
print(train.isna().any())
print('-'*10)
print(train.subtask_a.value_counts())
print(train.subtask_b.value_counts())
print(train.subtask_c.value_counts())

tweet_exp    False
subtask_a    False
subtask_b     True
subtask_c     True
dtype: bool
----------
NOT    8840
OFF    4400
Name: subtask_a, dtype: int64
TIN    3876
UNT     524
Name: subtask_b, dtype: int64
IND    2407
GRP    1074
OTH     395
Name: subtask_c, dtype: int64


In [0]:
def clean_tweet(text):
#     def handle_emoji(text):
#       # Dictionnary of "important" emojis : 
#       emoji_dict =  {'♥️': ' love ',
#                      '❤️' : ' love ',
#                      '❤' : ' love ',
#                      '😘' : ' kisses ',
#                     '😭' : ' cry ',
#                     '💪' : ' strong ',
#                     '🌍' : ' earth ',
#                     '💰' : ' money ',
#                     '👍' : ' ok ',
#                      '👌' : ' ok ',
#                     '😡' : ' angry ',
#                     '🍆' : ' dick ',
#                     '🤣' : ' haha ',
#                     '😂' : ' haha ',
#                     '🖕' : ' fuck you '}

#       for cha in emoji_dict:
#           text = re.compile(str(cha)).sub(str(emoji_dict[cha]),text)
#       # Remove ALL emojis
#       text = emoji.get_emoji_regexp().sub(r' ',text) 
#       text = re.compile("([\U0001f3fb-\U0001f3ff])").sub(r'',text) 
#       text = re.compile("([\U00010000-\U0010ffff])").sub(r'',text) 
#       text = re.compile("(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])").sub(r'',text)

#       # Add Space between  the Emoji Expressions : 
#       text = re.compile("([\U00010000-\U0010ffff])").sub(r' \1 ',text) 
#       return text
    ''' Function that is applied to every to tweet in the dataset '''

    # =========== TEXT ===========
    # Replace @USER by <user>
    text = re.compile(r'@USER').sub(r'<user>',text)

    # Replace URL by <url>
    text = re.compile(r'URL').sub(r'<url>',text)

    # Remove numbers :
    text = re.compile(r'[0-9]+').sub(r' ',text)

    # Remove some special characters
    text = re.compile(r'([\xa0_\{\}\[\]¬•$,:;/@#|\^*%().~`”"“-])').sub(r' ',text) 

    # Space the special characters with white spaces
    text = re.compile(r'([$&+,:;=?@#|.^*()%!"’“-])').sub(r' \1 ',text)

    # Replace some special characters : 
    replace_dict = {r'&' : 'and' , 
                    r'\+' : 'plus'}
    for cha in replace_dict:
        text = re.compile(str(cha)).sub(str(replace_dict[cha]),text)

    # Handle Emoji : translate some and delete the others
#     text = handle_emoji(text)

    # Word delengthening : 
    text = re.compile(r'(.)\1{3,}').sub(r'\1\1',text)

    # Cut the words with caps in them : 
    text = re.compile(r'([a-z]+|[A-Z]+|[A-Z][a-z]+)([A-Z][a-z]+)').sub(r'\1 \2',text)
    text = re.compile(r'([a-z]+|[A-Z]+|[A-Z][a-z]+)([A-Z][a-z]+)').sub(r'\1 \2',text)        
    # =========== TOKENS ===========
    # TOKENIZE 
    text = text.split(' ')

    # Remove white spaces tokens
    text = [text[i] for i in range(len(text)) if text[i] != ' ']

    # Remove empty tokens
    text = [text[i] for i in range(len(text)) if text[i] != '']

    # Remove repetition in tokens (!!! => !)
    text = [text[i] for i in range(len(text)) if text[i] != text[i-1]]

    #  Handle the ALL CAPS Tweets 
    ### if ratio of caps in the word > 75% add allcaps tag <allcaps>
    caps_r = np.mean([text[i].isupper() for i in range(len(text))])
    if caps_r > 0.6 : 
        text.append('<allcaps>')

    # Lower Case : 
    text = [text[i].lower() for i in range(len(text))]

    return ' '.join(text)

In [21]:
# clean_tweet('@USER @USER Neither do conservatives with May at the helm')


'<user> neither do conservatives with may at the helm'

In [113]:
import re, string
def _run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    def _is_punctuation(char):
        # print('[%s]+' % re.escape(string.punctuation))
        if re.match(r'[\!\"\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\[\\\]\^_\`\{\|\}\~]+',char):
            return True
        else:
            return False
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1
    text_list = ["".join(x) for x in output]
    text = ' '.join(text_list)
    text = text.split(' ')
    return ' '.join(text)
print(_run_split_on_punc("@USER @USER @USER @USER LOL!!!   Throwing the BULLSHIT Flag on such nonsense!!  #PutUpOrShutUp"))

@USER @USER @USER @USER LOL ! ! !    Throwing the BULLSHIT Flag on such nonsense ! !   #PutUpOrShutUp


In [0]:
train.tweet_exp = train.tweet_exp.apply(_run_split_on_punc)
train.tweet_exp = train.tweet_exp.apply(clean_tweet)
for subtask in task_list:
    data['test'][subtask].tweet_exp = data['test'][subtask].tweet_exp.apply(_run_split_on_punc)
    data['test'][subtask].tweet_exp = data['test'][subtask].tweet_exp.apply(clean_tweet)

In [35]:
train.loc[90194,'tweet_exp']

'<user> go home you are drunk ! <user> maga trump <url>'

In [115]:
classes_dict = {}
classes_dict['subtask_a'] = {'NOT' : 0 ,'OFF' : 1}
classes_dict['subtask_b'] = {'UNT' : 0 ,'TIN' : 1}
classes_dict['subtask_c'] = {'IND' : 0 ,'OTH' : 1, 'GRP' : 2}
for subtask in task_list:
    data['train'][subtask]=pd.DataFrame()
    for label in classes_dict[subtask]:
        data_label = train[train[subtask]==label]
        data['train'][subtask] = data['train'][subtask].append(train.loc[data_label.index])
    print(subtask,":")
    print(data['train'][subtask][subtask].value_counts())
    print(len(data['train'][subtask]))
    print('-'*10)

subtask_a :
NOT    8840
OFF    4400
Name: subtask_a, dtype: int64
13240
----------
subtask_b :
TIN    3876
UNT     524
Name: subtask_b, dtype: int64
4400
----------
subtask_c :
IND    2407
GRP    1074
OTH     395
Name: subtask_c, dtype: int64
3876
----------


In [116]:
import numpy as np
corpus_train = train.tweet_exp
corpus = pd.concat([train.tweet_exp, data['test']['subtask_a'].tweet_exp,data['test']['subtask_b'].tweet_exp,data['test']['subtask_c'].tweet_exp])
print(train.shape,corpus.shape)
print(data['test']['subtask_a'].shape,data['test']['subtask_b'].shape,data['test']['subtask_c'].shape)
pd.DataFrame(corpus.iloc[:5],columns=['tweet_exp'])

(13240, 4) (14553,)
(860, 1) (240, 1) (213, 1)


,tweet_exp
id,
86426,<user> she should ask a few native americans w...
90194,<user> go home you are drunk ! <user> maga tru...
16820,amazon is investigating chinese employees who ...
62688,<user> someone should've taken this piece of s...
43605,<user> obama wanted liberals and amp illegals ...


In [117]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS=10000
tokenizer = Tokenizer(nb_words=None,filters='')
tokenizer.fit_on_texts(corpus)
X = tokenizer.texts_to_sequences(corpus_train)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [118]:
for seq in X[:3]:
    print(seq)
    print([tokenizer.index_word[idx] for idx in seq])
MAX_SEQUENCE_LENGTH = max([len(X[i]) for i in range(len(X))])
print('MAX_SEQUENCE_LENGTH:',MAX_SEQUENCE_LENGTH)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

[1, 17, 80, 340, 7, 466, 2875, 303, 37, 67, 134, 24, 22, 3]
['<user>', 'she', 'should', 'ask', 'a', 'few', 'native', 'americans', 'what', 'their', 'take', 'on', 'this', 'is']
[1, 91, 308, 6, 8, 1043, 12, 1, 43, 63, 20]
['<user>', 'go', 'home', 'you', 'are', 'drunk', '!', '<user>', 'maga', 'trump', '<url>']
[1945, 3, 7009, 2072, 2646, 49, 8, 1755, 5503, 1455, 4, 1670, 130, 9658, 285, 19, 64, 2876, 15, 2, 2877, 5504, 20, 1945, 43, 317, 985, 475]
['amazon', 'is', 'investigating', 'chinese', 'employees', 'who', 'are', 'selling', 'internal', 'data', 'to', 'third', 'party', 'sellers', 'looking', 'for', 'an', 'edge', 'in', 'the', 'competitive', 'marketplace', '<url>', 'amazon', 'maga', 'kag', 'china', 'tcot']
MAX_SEQUENCE_LENGTH: 66
Found 19638 unique tokens.


In [119]:
import keras
from keras.preprocessing.sequence import pad_sequences
def text2id(tokenizer,texts_pd,max_seq_length=MAX_SEQUENCE_LENGTH):
  x_training = tokenizer.texts_to_sequences(texts_pd)
  x_training = keras.preprocessing.sequence.pad_sequences(x_training,maxlen=max_seq_length)
  for seq in x_training:
      assert len(seq) == max_seq_length
  print("all sequence length are ",max_seq_length)
  return x_training
for subtask in task_list:
  data['train'][subtask+'_token_id'] = text2id(tokenizer, data['train'][subtask].tweet_exp)

all sequence length are  66
all sequence length are  66
all sequence length are  66


In [120]:
# 將分類標籤對應到剛定義的數字
for subtask in task_list:
  data['train'][subtask+'_y'] = data['train'][subtask][subtask].apply(lambda x: classes_dict[subtask][x])
  print(data['train'][subtask+'_y'].value_counts())
  data['train'][subtask+'_y'] = np.asarray(data['train'][subtask+'_y']).astype('float32')
  print(data['train'][subtask+'_y'][:5])

0    8840
1    4400
Name: subtask_a, dtype: int64
[0. 0. 0. 0. 0.]
1    3876
0     524
Name: subtask_b, dtype: int64
[0. 0. 0. 0. 0.]
0    2407
2    1074
1     395
Name: subtask_c, dtype: int64
[0. 0. 0. 0. 0.]


In [41]:
for key in data.keys():
  print(key)
  print('*'*10)
  print(data[key].keys())
#   for sub_key in data[key].keys():
#     print(sub_key)

train
**********
dict_keys(['subtask_a', 'subtask_b', 'subtask_c', 'subtask_a_token_id', 'subtask_b_token_id', 'subtask_c_token_id', 'subtask_a_y', 'subtask_b_y', 'subtask_c_y'])
validation
**********
dict_keys([])
test
**********
dict_keys(['subtask_a', 'subtask_b', 'subtask_c'])


In [121]:
from sklearn.model_selection import train_test_split
NUM_LSTM_UNITS = 128
# y_train = keras.utils.to_categorical(y_train)

VALIDATION_RATIO = 0.15
# 小彩蛋
RANDOM_STATE = 9527
for subtask in task_list:
#   if subtask == 'subtask_c':
#     data['train'][subtask+'_y'] = keras.utils.to_categorical(data['train'][subtask+'_y'])
  data['train'][subtask+'_x_train'], data['validation'][subtask+'_x_val'], \
  data['train'][subtask+'_y_train'], data['validation'][subtask+'_y_val'] =train_test_split(data['train'][subtask+'_token_id'],data['train'][subtask+'_y'],test_size=VALIDATION_RATIO,random_state=RANDOM_STATE)
  print("Training Set",'----',subtask)
  print("-" * 10)
  print("x_train: ", data['train'][subtask+'_x_train'].shape)
  print("y_train : ", data['train'][subtask+'_y_train'].shape)

  print("-" * 10)
  print("x_val:   ", data['validation'][subtask+'_x_val'].shape)
  print("y_val :   ", data['validation'][subtask+'_y_val'].shape)
  print("#" * 10)

Training Set ---- subtask_a
----------
x_train:  (11254, 66)
y_train :  (11254,)
----------
x_val:    (1986, 66)
y_val :    (1986,)
##########
Training Set ---- subtask_b
----------
x_train:  (3740, 66)
y_train :  (3740,)
----------
x_val:    (660, 66)
y_val :    (660,)
##########
Training Set ---- subtask_c
----------
x_train:  (3294, 66)
y_train :  (3294,)
----------
x_val:    (582, 66)
y_val :    (582,)
##########


In [16]:
for key in data.keys():
  print(key,' ---------->')
  print(data[key].keys())
#   for sub_key in data[key].keys():
#     print(sub_key)
  print('*'*10)

train  ---------->
dict_keys(['subtask_a', 'subtask_b', 'subtask_c', 'subtask_a_token_id', 'subtask_b_token_id', 'subtask_c_token_id', 'subtask_a_y', 'subtask_b_y', 'subtask_c_y', 'subtask_a_x_train', 'subtask_a_y_train', 'subtask_b_x_train', 'subtask_b_y_train', 'subtask_c_x_train', 'subtask_c_y_train'])
**********
validation  ---------->
dict_keys(['subtask_a_x_val', 'subtask_a_y_val', 'subtask_b_x_val', 'subtask_b_y_val', 'subtask_c_x_val', 'subtask_c_y_val'])
**********
test  ---------->
dict_keys(['subtask_a', 'subtask_b', 'subtask_c'])
**********


In [17]:
GLOVE_DIR='./gdrive/My Drive/NLP/Final-Project/WordEmb'
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [18]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
null_words = open('./output/null-word.txt', 'w', encoding='utf-8')
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        null_words.write(word + '\n')
print('###[EMB] Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
###[EMB] Null word embeddings: 5696

###[EMB] Null word embeddings: 2710


In [0]:
for subtask in task_list:
  if subtask == 'subtask_c':
    data['train'][subtask+'_y'] = keras.utils.to_categorical(data['train'][subtask+'_y'])
  data['train'][subtask+'_x_train'], data['validation'][subtask+'_x_val'], \
  data['train'][subtask+'_y_train'], data['validation'][subtask+'_y_val'] =train_test_split(data['train'][subtask+'_token_id'],data['train'][subtask+'_y'],test_size=VALIDATION_RATIO,random_state=RANDOM_STATE)

In [129]:
a=np.where(data['train']['subtask_b'+'_y_train']==classes_dict["subtask_b"]['UNT'])
print(a[0].shape)

(444,)


In [130]:
def balanceData(subtask):
    ''' Augments the Data given in input in order to balance the dataset'''
    class_size = {}
    for label in classes_dict[subtask]:
        class_size[label] = len(data['train'][subtask+'_x_train'][data['train'][subtask+'_y_train']==classes_dict[subtask][label]])
        print(classes_dict[subtask][label])
    largest_class = max(class_size, key=class_size.get)
    print(largest_class)
    print('---- Augmenting the Data : ')
    print('Before Augmentation : ',class_size)

    for label in classes_dict[subtask]:  
        if label != largest_class:
            id_list = np.where(data['train'][subtask+'_y_train']==classes_dict[subtask][label])[0]
            nb_augmentation = class_size[largest_class] - class_size[label]
            id_augmentation = np.random.choice(id_list, (nb_augmentation,))
            data['train'][subtask+'_x_train']=np.append(data['train'][subtask+'_x_train'],data['train'][subtask+'_x_train'][id_augmentation],axis=0)
            data['train'][subtask+'_y_train']=np.append(data['train'][subtask+'_y_train'],data['train'][subtask+'_y_train'][id_augmentation],axis=0)
    # Check if it went well
    for label in classes_dict[subtask]:
        class_size[label] = len(data['train'][subtask+'_x_train'][data['train'][subtask+'_y_train']==classes_dict[subtask][label]])

    print('After Augmentation : ',class_size)
    return data
for subtask in task_list:
  balanceData(subtask)
#   balanceData(data['train'][subtask],subtask)
  class_size = {}
  for label in classes_dict[subtask]:
    class_size[label] = len(data['train'][subtask+'_y_train'][data['train'][subtask+'_y_train']==classes_dict[subtask][label]])
  print('**After Augmentation : ',class_size)
# # # 將分類標籤對應到剛定義的數字
# for subtask in task_list:
#   data['train'][subtask+'_y'] = data['train'][subtask][subtask].apply(lambda x: classes_dict[subtask][x])
#   print(data['train'][subtask+'_y'].value_counts())
#   data['train'][subtask+'_y'] = np.asarray(data['train'][subtask+'_y']).astype('float32')
#   print(data['train'][subtask+'_y'][:5])

0
1
NOT
---- Augmenting the Data : 
Before Augmentation :  {'NOT': 7542, 'OFF': 3712}
After Augmentation :  {'NOT': 7542, 'OFF': 7542}
**After Augmentation :  {'NOT': 7542, 'OFF': 7542}
0
1
TIN
---- Augmenting the Data : 
Before Augmentation :  {'UNT': 444, 'TIN': 3296}
After Augmentation :  {'UNT': 3296, 'TIN': 3296}
**After Augmentation :  {'UNT': 3296, 'TIN': 3296}
0
1
2
IND
---- Augmenting the Data : 
Before Augmentation :  {'IND': 2043, 'OTH': 334, 'GRP': 917}
After Augmentation :  {'IND': 2043, 'OTH': 2043, 'GRP': 2043}
**After Augmentation :  {'IND': 2043, 'OTH': 2043, 'GRP': 2043}


In [0]:
data['train']['subtask_c'+'_y_train'] = keras.utils.to_categorical(data['train']['subtask_c'+'_y_train'])
data['validation']['subtask_c'+'_y_val'] = keras.utils.to_categorical(data['validation']['subtask_c'+'_y_val'])

In [44]:
def balanceData(data,subtask):
    ''' Augments the Data given in input in order to balance the dataset'''
    class_size = {}
    for label in classes_dict[subtask]:
        class_size[label] = len(data[data[subtask]==label])
        print(classes_dict[subtask][label])
    largest_class = max(class_size, key=class_size.get)
    print(largest_class)
    print('---- Augmenting the Data : ')
    print('Before Augmentation : ',class_size)

    for label in classes_dict[subtask]:  
        if label != largest_class:
            id_list = data[data[subtask]==label].index
            nb_augmentation = class_size[largest_class] - class_size[label]
            id_augmentation = np.random.choice(id_list, (nb_augmentation,))
            data = data.append(data.loc[id_augmentation,:])
    # Check if it went well
    for label in classes_dict[subtask]:
        class_size[label] = len(data[data[subtask]==label])

    print('After Augmentation : ',class_size)
    return data
for subtask in task_list:
  data['train'][subtask] = balanceData(data['train'][subtask],subtask)
#   balanceData(data['train'][subtask],subtask)
  class_size = {}
  for label in classes_dict[subtask]:
    class_size[label] = len(data['train'][subtask][data['train'][subtask][subtask]==label])
  print('**After Augmentation : ',class_size)
# # 將分類標籤對應到剛定義的數字
for subtask in task_list:
  data['train'][subtask+'_y'] = data['train'][subtask][subtask].apply(lambda x: classes_dict[subtask][x])
  print(data['train'][subtask+'_y'].value_counts())
  data['train'][subtask+'_y'] = np.asarray(data['train'][subtask+'_y']).astype('float32')
  print(data['train'][subtask+'_y'][:5])

0
1
NOT
---- Augmenting the Data : 
Before Augmentation :  {'NOT': 8840, 'OFF': 4400}
After Augmentation :  {'NOT': 8840, 'OFF': 8840}
**After Augmentation :  {'NOT': 8840, 'OFF': 8840}
0
1
TIN
---- Augmenting the Data : 
Before Augmentation :  {'UNT': 524, 'TIN': 3876}
After Augmentation :  {'UNT': 3876, 'TIN': 3876}
**After Augmentation :  {'UNT': 3876, 'TIN': 3876}
0
1
2
IND
---- Augmenting the Data : 
Before Augmentation :  {'IND': 2407, 'OTH': 395, 'GRP': 1074}
After Augmentation :  {'IND': 2407, 'OTH': 2407, 'GRP': 2407}
**After Augmentation :  {'IND': 2407, 'OTH': 2407, 'GRP': 2407}
1    8840
0    8840
Name: subtask_a, dtype: int64
[0. 0. 0. 0. 0.]
1    3876
0    3876
Name: subtask_b, dtype: int64
[0. 0. 0. 0. 0.]
2    2407
1    2407
0    2407
Name: subtask_c, dtype: int64
[0. 0. 0. 0. 0.]


In [25]:
data['train']['subtask_a']['subtask_a'].value_counts()

NOT    8840
OFF    4400
Name: subtask_a, dtype: int64

In [0]:
from keras.optimizers import Adam
import pickle
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint

In [0]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,accuracy_score
class Metrics(Callback):
    def __init__(self, filepath,validation_data=(),IsBinaryClass=True):
        self.file_path = filepath
        self.X_val, self.y_val = validation_data
        if not IsBinaryClass:
          self.y_val = np.argmax(self.y_val,axis=1)
        self.IsBinaryClass_ = IsBinaryClass
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.best_val_f1 = 0
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        if self.IsBinaryClass_:
          val_predict = (np.asarray(self.model.predict(self.X_val))).round().astype('int').reshape(-1)
        else:
          val_predict = np.argmax(self.model.predict(self.X_val),axis=1)
        val_targ = self.y_val
        _val_f1 = f1_score(val_targ, val_predict,average='macro')
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
#         self.val_recalls.append(_val_recall)
#         self.val_precisions.append(_val_precision)
#         print(' — val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
        
        print(' — val_f1: %f' %(_val_f1))
        print("max f1; ",max(self.val_f1s))
        if _val_f1 > self.best_val_f1:
            self.model.save(self.file_path, overwrite=True)
            self.best_val_f1 = _val_f1
            print("best f1: {}".format(self.best_val_f1))
        else:
            print("val f1: {}, but not the best f1".format(_val_f1))
        return

In [0]:
def LSTM_model(IsBinaryClass=True):
  input_data = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
  emb_x = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              trainable=True)(input_data)
  merged = LSTM(100)(emb_x)

#   merged = Dense(units=128,kernel_initializer='random_normal')(merged)
#   merged=PReLU()(merged)
#   merged = Dropout(0.2)(merged)
  merged=BatchNormalization()(merged)
  if IsBinaryClass:
    predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal')(merged)
  else:
    predictions = Dense(3, activation='softmax', kernel_initializer='random_normal')(merged)

  model = Model(
      inputs=input_data,
      outputs=predictions)
  return model

In [0]:
def CNN_binary_class(IsBinaryClass=True):
  input_data = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
  emb_x = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              trainable=True)(input_data)
  conv1_x = Conv1D(100, 3, activation='relu')(emb_x)
  x_1 =GlobalMaxPooling1D()(conv1_x)
  conv2_x = Conv1D(100, 4, activation='relu')(emb_x)
  x_2 = GlobalMaxPooling1D()(conv2_x)
  conv3_x = Conv1D(100, 5, activation='relu')(emb_x)
  x_3 = GlobalMaxPooling1D()(conv3_x)

  merged = concatenate([x_1, x_2,x_3],axis=-1)
#   merged = Dropout(0.2)(merged)
  merged=BatchNormalization()(merged)
  merged = Dense(units=128,kernel_initializer='random_normal')(merged)
  merged=PReLU()(merged)
#   merged = Dropout(0.2)(merged)
  merged=BatchNormalization()(merged)
  if IsBinaryClass:
    predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal')(merged)
  else:
    predictions = Dense(3, activation='softmax', kernel_initializer='random_normal')(merged)

  model = Model(
      inputs=input_data,
      outputs=predictions)
  return model

TARGET_TASK = 'subtask_a'  ----->  train subtask a

In [0]:
# def MLP_model(IsBinaryClass=True):
#   input_data = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
#   emb_x = Embedding(len(word_index) + 1,
#                               EMBEDDING_DIM,
#                               weights=[embedding_matrix],
#                               input_length=MAX_SEQUENCE_LENGTH,
#                               trainable=True)(input_data)
#   emb_x = Dense(units=128,input_shape=(len(word_index) + 1,),kernel_initializer='random_normal')(emb_x)
#   emb_x=PReLU()(emb_x)
#   emb_x = Dense(units=64,kernel_initializer='random_normal')(emb_x)
#   emb_x=PReLU()(emb_x)
#   if IsBinaryClass:
#     predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal')(emb_x)
#   else:
#     predictions = Dense(3, activation='softmax', kernel_initializer='random_normal')(emb_x)

#   model = Model(
#       inputs=input_data,
#       outputs=predictions)
#   return model

In [276]:
TARGET_TASK = 'subtask_c'
# data['validation'][TARGET_TASK+'_y_val'] = np.argmax(data['validation'][TARGET_TASK+'_y_val'],axis=1)
Is_Binary_Class = True
if TARGET_TASK == 'subtask_c':
  Is_Binary_Class = False
MODEL_SAVE_PATH='./output/CNN_Train_'+TARGET_TASK+'.hdf5' 
metrics = Metrics(MODEL_SAVE_PATH,
                  validation_data=(data['validation'][TARGET_TASK+'_x_val'],data['validation'][TARGET_TASK+'_y_val']),
                  IsBinaryClass=Is_Binary_Class)

model=LSTM_model(IsBinaryClass=Is_Binary_Class)
model.summary()
# adam1 = keras.optimizers.Adam(lr=0.0001,decay=0.01) #for CNN model
adam1 = keras.optimizers.Adam(lr=0.001,decay=0.0001) #for LSTM model
model.compile(loss='binary_crossentropy',
          optimizer= adam1,
          metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 66)                0         
_________________________________________________________________
embedding_24 (Embedding)     (None, 66, 100)           1963900   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
batch_normalization_45 (Batc (None, 100)               400       
_________________________________________________________________
dense_45 (Dense)             (None, 3)                 303       
Total params: 2,045,003
Trainable params: 2,044,803
Non-trainable params: 200
_________________________________________________________________


In [265]:
BATCH_SIZE = 50
NUM_EPOCHS = 5
class_weight = {0: 1.,
                1: 2.}
# class_weight = {0: 7.3,
#                 1: 1.}
# class_weight = {0: 1.,
#                 1: 6.1,
#                 2: 2.24
#                }
history = model.fit(
    x=data['train'][TARGET_TASK+'_x_train'],
    y=data['train'][TARGET_TASK+'_y_train'],
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[metrics],
    # 每個 epoch 完後計算驗證資料集
    # 上的 Loss 以及準確度
    validation_data=(data['validation'][TARGET_TASK+'_x_val'],data['validation'][TARGET_TASK+'_y_val']),
    # 每個 epoch 隨機調整訓練資料集
    # 裡頭的數據以讓訓練過程更穩定
    shuffle=True
)
#     class_weight=class_weight

Train on 6592 samples, validate on 660 samples
Epoch 1/5
6592/6592 [==============================] - 28s 4ms/step - loss: 0.5601 - acc: 0.7081 - val_loss: 0.4535 - val_acc: 0.7985
 — val_f1: 0.596434
max f1;  0.5964342033276478
best f1: 0.5964342033276478
Epoch 2/5
6592/6592 [==============================] - 15s 2ms/step - loss: 0.3025 - acc: 0.8762 - val_loss: 0.8172 - val_acc: 0.6636
 — val_f1: 0.527787
max f1;  0.5964342033276478
val f1: 0.5277874463394825, but not the best f1
Epoch 3/5
6592/6592 [==============================] - 15s 2ms/step - loss: 0.1168 - acc: 0.9612 - val_loss: 0.6947 - val_acc: 0.8667
 — val_f1: 0.584145
max f1;  0.5964342033276478
val f1: 0.5841448046740749, but not the best f1
Epoch 4/5
6592/6592 [==============================] - 15s 2ms/step - loss: 0.0531 - acc: 0.9826 - val_loss: 0.7744 - val_acc: 0.8439
 — val_f1: 0.580554
max f1;  0.5964342033276478
val f1: 0.58055420155364, but not the best f1
Epoch 5/5
6592/6592 [==============================] - 

In [0]:
file_his = open('./output/CNNV5_train_hist_'+TARGET_TASK+'lstm_1.pickle', 'wb')
pickle.dump(history.history, file_his)
file_his.close()

In [284]:
history.history.keys()
history.history['f-score']=metrics.val_f1s
# history.history['best-fscore']=metrics.val_f1s
history.history.keys()

AttributeError: ignored

In [277]:
data['test'][TARGET_TASK+'_x_test'] = text2id(tokenizer, data['test'][TARGET_TASK].tweet_exp)

all sequence length are  66


In [278]:
from keras.models import load_model
if os.path.exists(MODEL_SAVE_PATH):
      model=load_model(MODEL_SAVE_PATH)
    # 若成功加载前面保存的参数，输出下列信息
      print("checkpoint_loaded")
predictions = model.predict(data['test'][TARGET_TASK+'_x_test'])

checkpoint_loaded


In [279]:
#run this block if TARGET_TASK = 'subtask_c'
predictions = np.argmax(predictions,axis=1)
predictions[:5]

array([2, 1, 0, 0, 0])

In [267]:
#run this block if TARGET_TASK = 'subtask_a' or 'subtask_b'
predictions=predictions.round().astype('int').reshape(-1)
predictions[:5]


array([1, 1, 0, 1, 0])

In [280]:
TEST_A_LABEL_PATH = DATA_SET_DIR+'labels-levela.csv'
TEST_B_LABEL_PATH = DATA_SET_DIR+'labels-levelb.csv'
TEST_C_LABEL_PATH = DATA_SET_DIR+'labels-levelc.csv'
TEST_LABEL_PATH_DICT = {}
TEST_LABEL_PATH_DICT['subtask_a'] = TEST_A_LABEL_PATH
TEST_LABEL_PATH_DICT['subtask_b'] = TEST_B_LABEL_PATH
TEST_LABEL_PATH_DICT['subtask_c'] = TEST_C_LABEL_PATH

true_label = pd.read_csv(TEST_LABEL_PATH_DICT[TARGET_TASK],sep=',',header=None)
true_label.head(3)

,0,1
0,15923,OTH
1,60133,GRP
2,83681,IND


In [281]:
y_test = true_label[1].apply(lambda x: classes_dict[TARGET_TASK][x])
y_test = np.asarray(y_test).astype('int')
y_test[:5]

array([1, 2, 0, 1, 0])

In [282]:
testing_f1 = f1_score(y_test, predictions,average='macro')
testing_acc = accuracy_score(y_test, predictions)
print('testing data f1-score:',testing_f1)
print('testing data accuracy:',testing_acc)
# testing data f1-score: 0.4489635185344832
# testing data accuracy: 0.49765258215962443
# testing data f1-score: 0.5304134439163793
# testing data accuracy: 0.6150234741784038

testing data f1-score: 0.5773644118175659
testing data accuracy: 0.6291079812206573


In [285]:
!zip cnn_outputt3_lstm_1 output/*

  adding: output/CNN_Train_subtask_a.hdf5 (deflated 15%)
  adding: output/CNN_Train_subtask_b.hdf5 (deflated 41%)
  adding: output/CNN_Train_subtask_c.hdf5 (deflated 42%)
  adding: output/CNNV5_train_hist_subtask_a.pickle (deflated 42%)
  adding: output/CNNV5_train_hist_subtask_b_2.pickle (deflated 43%)
  adding: output/CNNV5_train_hist_subtask_blstm_1.pickle (deflated 22%)
  adding: output/CNNV5_train_hist_subtask_c_1.pickle (deflated 40%)
  adding: output/null-word.txt (deflated 48%)


In [0]:
!mv cnn_outputt3_lstm_1.zip './gdrive/My Drive/NLP/Final-Project/output/'

In [0]:
!mv output/CNNV5_train_hist_subtask_b.pickle output/CNNV5_train_hist_subtask_b_2.pickle

In [0]:
from tqdm import tqdm_notebook